In [ ]:
# ============================================================
# Free–Free Brightness Temperature Calculation 
# Simpson's Integration Rule
# ============================================================

import numpy as np


# Free–free coefficient (from paper)
CFF = 9.786e-3


def coulomb_log(Te, nu):
    """
    Coulomb logarithm
    Te : Electron temperature [K]
    nu : Frequency [Hz]
    """
    return 24.5 + np.log(Te) - np.log(nu)


def alpha_ff(ne, Te, nu):
    """
    Free-free absorption coefficient alpha_ff(z, nu)
    """
    lnL = coulomb_log(Te, nu)
    return CFF * ne**2 * lnL / (nu**2 * Te**1.5)


def simpson_integral(y, x):
    """
    Simpson's 1/3 integration rule
    Requires odd number of grid points
    """
    n = len(x)
    if n % 2 == 0:
        raise ValueError("Simpson rule requires an odd number of points")

    h = (x[-1] - x[0]) / float(n - 1)

    S = y[0] + y[-1]
    S += 4.0 * np.sum(y[1:-1:2])
    S += 2.0 * np.sum(y[2:-2:2])

    return S * h / 3.0


def optical_depth(z, ne, Te, nu):
    """
    Optical depth τ_ff(z, nu) using Simpson integration
    """
    alpha = alpha_ff(ne, Te, nu)
    tau = np.zeros_like(z)

    for i in range(2, len(z), 2):
        tau[i] = simpson_integral(alpha[:i+1], z[:i+1])
        tau[i-1] = tau[i]  # fill intermediate point

    return tau


def brightness_temperature(z, ne, Te, nu):
    """
    Brightness temperature T_B(nu)
    """
    alpha = alpha_ff(ne, Te, nu)
    tau = optical_depth(z, ne, Te, nu)
    integrand = Te * np.exp(-tau) * alpha

    TB = simpson_integral(integrand, z)
    return TB


# ============================================================
# Example coronal model
# ============================================================

if __name__ == "__main__":

    # Line-of-sight grid (must be odd number of points!)
    z = np.linspace(-7.0e10, 0.0, 3001)  # cm

    # Electron density profile [cm^-3]
    ne0 = 1.0e8
    scale_height = 5.0e9
    ne = ne0 * np.exp(z / scale_height)

    # Electron temperature profile [K]
    Te = 1.5e6 * np.ones_like(z)

    # Observing frequency
    nu = 150.0e6  # Hz

    TB = brightness_temperature(z, ne, Te, nu)

    print("======================================")
    print("Free–Free Brightness Temperature")
    print("Frequency        : %.1f MHz" % (nu / 1e6))
    print("Brightness Temp. : %.3e K" % TB)
    print("======================================")


Free–Free Brightness Temperature
Frequency        : 150.0 MHz
Brightness Temp. : 1.666e+05 K
